In [29]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [30]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [31]:
data_path = ''

In [32]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 128
    lr = 0.001
    # epochs = 200
    epochs = 100
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [33]:
hist_file = get_hist_file('histograms.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

/Users/ruiqiwang/Desktop/QueryFormer/QueryFormer/model/database_util.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['freq'][i] = freq_np
/Users/ruiqiwang/Desktop/QueryFormer/QueryFormer/model/database_util.py:89: FutureWarnin

In [34]:
encoding_ckpt = torch.load('checkpoints/tpch_encoding.pt')
encoding = encoding_ckpt['encoding']
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')

/var/folders/6w/0yh1f35j21x97nd30qwjxn900000gn/T/ipykernel_3770/2872170053.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoding_ckpt = torch.load('checkpoints/tpch_e

In [35]:
from model.util import seed_everything
seed_everything()

In [36]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )

62


In [37]:
_ = model.to(args.device)

In [38]:
to_predict = 'cost'

In [39]:
imdb_path = ''
dfs = []  # list to hold DataFrames
# SQ: added

file = 'query_plans.csv'
df = pd.read_csv(file)
dfs.append(df)

full_train_df = pd.concat(dfs)

val_dfs = []  # list to hold DataFrames

file = 'query_plans.csv'
df = pd.read_csv(file)
val_dfs.append(df)

val_df = pd.concat(val_dfs)

In [40]:
table_sample = get_job_table_sample('tpch')

Loaded queries with len  2
Loaded bitmaps


In [41]:
train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

val: 0.05, mini: 0.0, maxi: 0.1
Types: val=<class 'float'>, mini=<class 'numpy.float64'>, maxi=<class 'numpy.float64'>
dict_keys(['lineitem'])
lineitem
dict_keys(['orders', 'lineitem', 'customer'])
lineitem
dict_keys(['orders', 'lineitem', 'customer'])
orders
val: 185395923467803688847143, mini: 308527869792217406721093, maxi: 1425101975591223382617
Types: val=<class 'int'>, mini=<class 'int'>, maxi=<class 'int'>
dict_keys(['orders', 'lineitem', 'customer'])
customer
val: 0.05, mini: 0.0, maxi: 0.1
Types: val=<class 'float'>, mini=<class 'numpy.float64'>, maxi=<class 'numpy.float64'>
dict_keys(['lineitem'])
lineitem
dict_keys(['orders', 'lineitem', 'customer'])
lineitem
dict_keys(['orders', 'lineitem', 'customer'])
orders
val: 185395923467803688847143, mini: 308527869792217406721093, maxi: 1425101975591223382617
Types: val=<class 'int'>, mini=<class 'int'>, maxi=<class 'int'>
dict_keys(['orders', 'lineitem', 'customer'])
customer


In [42]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, cost_norm, args)

Epoch: 0  Avg Loss: 0.03980080410838127, Time: 0.08248710632324219
Median: 89.35714707200606
Mean: 89.35714707200606


Epoch: 20  Avg Loss: 5.4954296501819044e-05, Time: 1.2314953804016113
Median: 1.1684420868140952
Mean: 1.1684420868140952
Epoch: 40  Avg Loss: 3.3325744880130515e-05, Time: 1.8684160709381104
Median: 1.1009901762008667
Mean: 1.1009901762008667
Epoch: 60  Avg Loss: 2.058055906672962e-05, Time: 2.840327262878418
Median: 1.084700564183377
Mean: 1.084700564183377
Epoch: 80  Avg Loss: 1.2720599897875218e-06, Time: 3.80352520942688
Median: 1.0196996652175474
Mean: 1.0196996652175474


Model saved to ./checkpoints/tpch_model.pt


'6195933034023009986.pt'

In [44]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [48]:
import json

def print_plan(plan):
    # pretty print the json object

    # the following code parses the json string into a dictionary
    json_parsed = json.loads(plan)
    json_pretty = json.dumps(json_parsed, indent=4)
    print(json_pretty)

    with open('output.json', 'w') as f:
        f.write(json_pretty)



In [49]:
def print_qerror(preds_unnorm, labels_unnorm):
    qerror = []
    for i in range(len(preds_unnorm)):
        print("Predicted: {}, Actual: {}".format(preds_unnorm[i], labels_unnorm[i]))
        if preds_unnorm[i] > float(labels_unnorm[i]):
            qerror.append(preds_unnorm[i] / float(labels_unnorm[i]))
        else:
            qerror.append(float(labels_unnorm[i]) / float(preds_unnorm[i]))

    e_50, e_90 = np.median(qerror), np.percentile(qerror,90)    
    e_mean = np.mean(qerror)
    print("Median: {}".format(e_50))
    print("90th percentile: {}".format(e_90))
    print("Mean: {}".format(e_mean))
    return 

def get_corr(ps, ls): # unnormalised
    ps = np.array(ps)
    ls = np.array(ls)
    corr, _ = pearsonr(np.log(ps), np.log(ls))
    
    return corr

In [50]:
def evaluate(model, ds, bs, norm, device):
    model.eval()
    cost_predss = np.empty(0)

    with torch.no_grad():
        for i in range(0, len(ds), bs):
            batch, batch_labels = collator(list(zip(*[ds[j] for j in range(i,min(i+bs, len(ds)) ) ])))


            batch = batch.to(device)

            cost_preds, _ = model(batch)

            cost_preds = cost_preds.squeeze()

            cost_predss = np.append(cost_predss, cost_preds.cpu().detach().numpy())


    print_qerror(norm.unnormalize_labels(cost_predss), ds.costs)

    if len(cost_predss) > 2:  
        corr = get_corr(norm.unnormalize_labels(cost_predss), ds.costs)
        print('Corr: ', corr)

    return 

In [51]:
def eval_workload(workload, methods):

    get_table_sample = methods['get_sample']

    workload_file_name = 'tpch'
    output_file_name = '{}_output.csv'.format(workload)

    table_sample = get_table_sample(workload_file_name)

    plan_df = pd.read_csv('query_plans.csv')
    print_plan(plan_df['json'][0])
    workload_csv = pd.read_csv('tpch.csv',sep='#',header=None)
    workload_csv.columns = ['table','join','predicate','cardinality']

    workload_csv.to_csv(output_file_name, index=False)

    
    ds = PlanTreeDataset(plan_df, workload_csv, \
        methods['encoding'], methods['hist_file'], methods['cost_norm'], \
        methods['cost_norm'], 'cost', table_sample)
    
    

    evaluate(methods['model'], ds, methods['bs'], methods['cost_norm'], methods['device'])
    return 

In [52]:
eval_workload('tpch', methods)

Loaded queries with len  2
Loaded bitmaps
{
    "Plan": {
        "Node Type": "Aggregate",
        "Strategy": "Sorted",
        "Partial Mode": "Finalize",
        "Parallel Aware": false,
        "Async Capable": false,
        "Startup Cost": 184469.24,
        "Total Cost": 184471.2,
        "Plan Rows": 6,
        "Plan Width": 236,
        "Actual Startup Time": 3645.51,
        "Actual Total Time": 3646.542,
        "Actual Rows": 4,
        "Actual Loops": 1,
        "Group Key": [
            "l_returnflag",
            "l_linestatus"
        ],
        "Plans": [
            {
                "Node Type": "Gather Merge",
                "Parent Relationship": "Outer",
                "Parallel Aware": false,
                "Async Capable": false,
                "Startup Cost": 184469.24,
                "Total Cost": 184470.64,
                "Plan Rows": 12,
                "Plan Width": 236,
                "Actual Startup Time": 3645.496,
                "Actual Total 